In [1]:
import pandas as pd
import json
from urllib.request import urlopen

In [2]:
def getCharNames() -> dict:
    # Ensure arg types
    
    url = 'https://raw.githubusercontent.com/aizawey479/ba-data/jp/Excel/CharacterAcademyTagsExcelTable.json'
    response = urlopen(url)
    data = json.loads(response.read())['DataList']

    result = {}
    for node in data:
        name = next(tag for tag in node["FavorItemUniqueTags"] if tag.startswith("F_"))
        name = name[2:].lower()

        # Correct the name
        name = name.replace("zunko", "junko")
        name = name.replace("hihumi", "hifumi")

        components = name.split("_")
        result[node["Id"]] = components[0].capitalize()
        if len(components) > 1:
            if not (len(components) == 2 and components[1] == "default"):
                result[node["Id"]] += " ({})".format(" ".join(components[1:]))

    return result

In [3]:
def getBondStats():
    url = "https://raw.githubusercontent.com/aizawey479/ba-data/jp/Excel/FavorLevelRewardExcelTable.json"
    response = urlopen(url)
    df = pd.read_json(response)
    df = pd.json_normalize(df['DataList'])
    
    return df

In [4]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 

    result = (str1.join(map(str,s)))
    return result

In [5]:
def getBondBaseTable():
    charnames = getCharNames()
    bond_df = getBondStats()

    bond_df['StatType'] = bond_df['StatType'].apply(lambda x: listToString(x))
    bond_df = bond_df.merge(bond_df['StatType'].str.split(' ', expand=True), how='inner', left_index=True, right_index=True)

    bond_df['StatValue'] = bond_df['StatValue'].apply(lambda x: listToString(x))
    bond_df = bond_df.merge(bond_df['StatValue'].str.split(' ', expand=True), how='inner', left_index=True, right_index=True)
    bond_df['Charactername'] = bond_df['CharacterId'].map(charnames)

    column_dict = {
        'Charactername':'CharacterName',
        'CharacterId':'CharacterId',
        'FavorLevel':'Favorlevel',
        '0_x':'Stat1',
        '1_x':'Stat2',
        '0_y':'Value1',
        '1_y':'Value2'
    }

    bond_df = bond_df.rename(columns=column_dict)
    bond_df = bond_df[list(column_dict.values())]
    
    return bond_df

In [6]:
bond_df = getBondBaseTable()

In [7]:
bond_df

,CharacterName,CharacterId,Favorlevel,Stat1,Stat2,Value1,Value2
0,Aru,10000,2,AttackPower_Base,None,4,None
1,Aru,10000,3,AttackPower_Base,None,4,None
2,Aru,10000,4,AttackPower_Base,None,4,None
3,Aru,10000,5,AttackPower_Base,None,4,None
4,Aru,10000,6,AttackPower_Base,None,6,None
...,...,...,...,...,...,...,...
4209,Nodoka,26006,46,AttackPower_Base,MaxHP_Base,3,22
4210,Nodoka,26006,47,AttackPower_Base,MaxHP_Base,3,22
4211,Nodoka,26006,48,AttackPower_Base,MaxHP_Base,3,22
4212,Nodoka,26006,49,AttackPower_Base,MaxHP_Base,3,22
